# A Gentle introduction to Ray datasets APIs

© 2019-2022, Anyscale. All Rights Reserved

📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
⬅️ [Previous notebook](./ex_06_ray_api_calls.ipynb) <br>

### Overview

As a library built atop Ray, Ray data allows you to: 
 * exchange data among Ray tasks, actors, libraries, and applications.
 * read/write training data from different file sources and storage
 * supports myriad [file formats and data sources](https://docs.ray.io/en/latest/data/dataset.html#datasource-compatibility).
 * use [Apache Arrow](https://arrow.apache.org/) to load and preprocess data for distributed ML training pipelines.
 * transformations like `map`, `filter`, and `partition`. 

**Note**: Ray datasets is not a replacement for a full-fledged data processing library for doing exploratory data analysis (EDA), extract, transform and load (ETL) or a subsitute for Apache Spark or Dask or Pandas DataFrames. Its primary objective is the last-mile rudimentary and efficient distributed data preprocessing and data ingestion for ML training by providing simple primitive transformational functions.

<img src="images/dataset.png" width="75%" height="45%">

### Learning objectives

In this introductory tutorial you will learn:
 * what are Ray datasets, why use them, and when.
 * create, transform, read, and save Ray datasets
 * use shards for parallel processing of large datasets
 * use datasets for last-mile ML ingestion for distributed training
 
 **Note**: Even though you will get a deep dive into Ray data here, most of its functionality is implemented and subsumed in Ray AIR, which you'll hear about it tomorrow in the keynotes
 and few break out sessions.

### Key concepts

To work with Ray Datasets, you need to understand how `Datasets` and `Dataset Pipelines` work. A quick peek into each of these will shed some light into overall benefits of Ray Datasets.

Let's start with the internal format. 

#### Ray Datasets

A Ray dataset implements a distributed [Apache Arrow](https://arrow.apache.org/). As such, a Dataset consists of a *list of Ray object references* to blocks on the *shared memory object store*. Each block holds a set of items in either an [Arrow table](https://arrow.apache.org/docs/python/data.html#tables) (when created from or transformed to tabular or tensor data), a Pandas DataFrame (when created from or transformed to Pandas data), or a Python list (otherwise).

<img src="images/dataset-arch.png" width="60%" height="30%">

#### Dataset Pipelines
Datasets execute their transformations synchronously or eagerly in blocking calls. However, it can be useful to overlap dataset computations with output. This can be done with a `DatasetPipeline`.

A `DatasetPipeline` is a unified iterator over a sequence of Ray Datasets, each of which represents a window over the original data. Conceptually, it is similar to a `Spark DStream`, but manages execution over a bounded source of data instead of an unbounded stream. Ray computes each dataset window on-demand and stitches their output together into a single logical data iterator. `DatasetPipeline` implements most of the same transformation and output methods as Datasets (e.g., `map`, `filter`, `split`, `iter_rows`, `to_torch`, etc.).

<img src="images/ray_dataset_pipeline.jpg" width="60%" height="30%">

#### Common use case for Dataset Pipeline: Saturate your GPU resources


<img src="images/gpu_saturation.png" width="60%" height="30%">

### Datasets Execution Model
In this section, we briefly discuss the execution model of Datasets, which is useful for understanding and tuning performance.

#### Reading Data
Datasets uses Ray tasks, for parallelism, to read data from remote storage or source. When reading from a file-based datasource (e.g., S3, GCS), it creates a number of parallel
read tasks equal to the specified read parallelism. You can further [fine tune parallelism](https://docs.ray.io/en/master/data/performance-tips.html#tuning-read-parallelism).
One or more files will be assigned to each read task. Each read task reads its assigned files and produces one or more output blocks (Ray objects):

<img src="https://docs.ray.io/en/master/_images/dataset-read.svg" height="25%" width="50%">

In the common case, each read task produces a single output block. Read tasks may split the output into multiple blocks if the data exceeds the target max block size (2GiB by default). This automatic block splitting avoids out-of-memory errors when reading very large single files (e.g., a 100-gigabyte CSV file). 

#### Deferred Read Task Execution

When a Dataset is created using `ray.data.read_*`, only the first read is executed initially. This avoids blocking Dataset creation on the reading of all data files, enabling inspection functions like `ds.schema()` without incurring high read costs. `<ray.data.Dataset.schema>`() and `ds.show()` can be used right away. Executing further transformations on the Dataset will trigger execution of all read tasks.

#### Dataset Transforms

Datasets use either Ray tasks or Ray actors to transform datasets (i.e., for `ds.map_batches()`, `ds.map()`, or `ds.flat_map()`). By default, tasks are used `(compute="tasks")`. Actors can be specified with `compute="actors"`, in which case an autoscaling pool of Ray actors will be used to apply transformations. Use actors for expensive state initialization (e.g., for GPU-based tasks) that can be re-used. 

Whichever compute strategy is used, each map task generally takes in one block and produces one or more output blocks. The output block splitting rule is the same as for file reads (blocks are split after hitting the target max block size of 2GiB):

<img src="https://docs.ray.io/en/master/_images/dataset-map.svg" height="25%" width="50%">

Use `ActorStrategyPool`when workers take time to initialize, e.g., GPU memory or initialize certain values within an actor that could be resused again.


#### Shuffling Data

Certain operations like `ds.sort()` and `ds.groupby()` require data blocks to be partitioned by value.

You can also change the partitioning of a Dataset using `ds.random_shuffle()` or `ds.repartition()`. Use the former if you want to randomize the order of elements in the dataset; use the second if you only want to equalize the size of the Dataset blocks (e.g., after a read or transformation that may skew the distribution of block sizes). 

Note that repartition has two modes, `shuffle=False`, which performs the minimal data movement needed to equalize block sizes, and `shuffle=True`, which performs a full (non-random) distributed shuffle:

<img src="https://docs.ray.io/en/master/_images/dataset-shuffle.svg" height="25%" width="50%">

#### Fault tolerance

Datasets relies on task-based 🤹‍♀️ [fault tolerance](https://docs.ray.io/en/latest/ray-core/tasks/fault-tolerance.html) in Ray core. Specifically, a `Dataset` will be automatically recovered by Ray in case of failures. This works through **lineage reconstruction**: a Dataset is a collection of Ray objects stored in shared memory, and if any of these objects are lost, then Ray will recreate them by re-executing the task(s) that created them.

#### Execution and Memory Management

See [Execution and Memory Management](https://docs.ray.io/en/master/data/dataset-internals.html#) for more advanced details about how Datasets manages memory and optimizations such as [lazy vs eager execution](https://docs.ray.io/en/master/data/dataset-internals.html#execution). 

In [1]:
import logging, os, random, warnings
import ray

In [2]:
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
if ray.is_initialized:
    ray.shutdown()
ray.init(logging_level=logging.ERROR)

Python version:,3.8.13
Ray version:,2.0.0rc1
Dashboard:,http://127.0.0.1:8266


### Creating a simple Ray Dataset

For quick illustration, let's create a generic dataset of 1K integers and look at the schema and underlying datatype. [Ray Data API Documentation](https://docs.ray.io/en/latest/data/package-ref.html).

In [4]:
ds = ray.data.range(1000)
ds.count()

1000

In [5]:
ds.schema()

int

The difference between `show` and `take` is that the former takes one item at time and prints it, while the latter iterates over row items from the dataset, appends to a list and returns it. Underneath, `ds.show()` calls `ds.take()`.

In [6]:
ds.show(5)

0
1
2
3
4


In [7]:
ds.take(5)

[0, 1, 2, 3, 4]

### Creating a large Ray Dataset

Let's create a synthetic dataset, *Homeowners*, of Arrow records (800K) with several columns and data associated with it. 

To illustrate some simple transformational functions, we'll use this *Homeowners* generated data.

In [8]:
def get_anonomized_name(num_letters=6):
    import string
    return ("anonomized-" + "".join(random.choices(string.ascii_uppercase + string.ascii_lowercase, k=num_letters)))

def get_anonomized_ssn():
    return ("anonomized-" + "".join(["{}".format(random.randint(0, 9)) for num in range(1, 10)]))

In [9]:
NUM_ROWS = 800001
STATES = ["CA", "AZ", "OR", "WA", "TX", "UT", "NV", "NM"]
M_STATUS = ["married", "single", "domestic", "divorced", "undeclared"]
GENDER = ["F", "M", "U"]
HOME_OWNER = ["condo", "house", "rental", "cottage"]

items = [{"id": i,
          "ssn": get_anonomized_ssn(),
          "name": get_anonomized_name(),
          "amount": i * 1.5 * 10, 
          "interest": random.randint(1,5) * .1,
          "state": random.choice(STATES),
          "marital_status": random.choice(M_STATUS),
          "property": random.choice(HOME_OWNER),
          "dependents": random.randint(1, 5),
          "defaulted": random.randint(0,1),
          "gender":random.choice(GENDER) } for i in range(1,NUM_ROWS)]
items[:1]

[{'id': 1,
  'ssn': 'anonomized-173094288',
  'name': 'anonomized-GmLFLG',
  'amount': 15.0,
  'interest': 0.2,
  'state': 'AZ',
  'marital_status': 'domestic',
  'property': 'rental',
  'dependents': 5,
  'defaulted': 0,
  'gender': 'U'}]

#### Creating a dataset from list of dictionary items

Ray data can be created from a dictionary of items. 

*Question: how is num_blocks computed?*

The default number of blocks is 200, just like in Spark, the default number of partitions created when reading data is 200. This can be [tuned as need](https://docs.ray.io/en/master/data/performance-tips.html#tuning-read-parallelism). 

In [10]:
arrow_ds = ray.data.from_items(items)
arrow_ds

Dataset(num_blocks=200, num_rows=800000, schema={id: int64, ssn: string, name: string, amount: double, interest: double, state: string, marital_status: string, property: string, dependents: int64, defaulted: int64, gender: string})

In [11]:
arrow_ds.count()

800000

In [12]:
arrow_ds.take(1)

[ArrowRow({'id': 1,
           'ssn': 'anonomized-173094288',
           'name': 'anonomized-GmLFLG',
           'amount': 15.0,
           'interest': 0.2,
           'state': 'AZ',
           'marital_status': 'domestic',
           'property': 'rental',
           'dependents': 5,
           'defaulted': 0,
           'gender': 'U'})]

In [13]:
arrow_ds.schema()

id: int64
ssn: string
name: string
amount: double
interest: double
state: string
marital_status: string
property: string
dependents: int64
defaulted: int64
gender: string

### Saving datasets and reading as a parquet files 🗃
Ray datasets support myriad data formats. Let's save this dataset as a parquet file and create `N` partitions, where N=5.

In [14]:
path = os.path.abspath("data_homeowners/interest.parquet")
arrow_ds.repartition(5).write_parquet(path)

Write Progress: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 23.48it/s]


In [15]:
!ls -l data_homeowners/interest.parquet

total 53840
-rw-r--r--  1 jules  staff  5418325 Aug 14 15:13 78b07d2926b04d0e8064b503a297dde8_000000.parquet
-rw-r--r--  1 jules  staff  5534059 Aug 14 15:13 78b07d2926b04d0e8064b503a297dde8_000001.parquet
-rw-r--r--  1 jules  staff  5533367 Aug 14 15:13 78b07d2926b04d0e8064b503a297dde8_000002.parquet
-rw-r--r--  1 jules  staff  5533930 Aug 14 15:13 78b07d2926b04d0e8064b503a297dde8_000003.parquet
-rw-r--r--  1 jules  staff  5534231 Aug 14 15:13 78b07d2926b04d0e8064b503a297dde8_000004.parquet


### Reading data from parquet files

Next, we read a few of the files from the dataset. This read is semi-lazy, where reading of the first file is eagerly executed, but reading of all other files is delayed until the underlying data is needed by downstream operations (e.g., consuming the data with `ds.take()`, or transforming the data with `ds.map_batches())`.

In [16]:
arrow_ds = ray.data.read_parquet(path)

We can easily inspect the schema of this dataset. For Parquet files, we don’t even have to read the actual data to get the schema or take first rows; we can read it from the lightweight Parquet metadata!

In [17]:
arrow_ds.schema()

id: int64
ssn: string
name: string
amount: double
interest: double
state: string
marital_status: string
property: string
dependents: int64
defaulted: int64
gender: string

In [18]:
arrow_ds.take(1)

[ArrowRow({'id': 1,
           'ssn': 'anonomized-173094288',
           'name': 'anonomized-GmLFLG',
           'amount': 15.0,
           'interest': 0.2,
           'state': 'AZ',
           'marital_status': 'domestic',
           'property': 'rental',
           'dependents': 5,
           'defaulted': 0,
           'gender': 'U'})]

### Transforming data with simple methods

Ray datasets support transformation in parallel using `map`. It uses Ray tasks to execute eagerly or synchronously. Among others [transformations](https://docs.ray.io/en/latest/data/package-ref.html#dataset-api), it supports`filter`, `flat_map`, `groupBy`etc.

Let's try a using `.map()`, `.filter()` and `.groupBy` on our dataset. 

The `map()` and `filter()` are row-based operations. This can be expensive for large datasets. However, you can use `map_batches(...)` with batch_size=4096 as default. This will create a task per block and each batch will be vectorized and executed in parallel. Ray tasks are created per block for a map operation. 

Let's try first with row-based transformation

In [19]:
%%time
arrow_ds.filter(lambda x: x['amount'] > 10000).take(1)

Read->Filter: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.13it/s]

CPU times: user 20.5 ms, sys: 12.7 ms, total: 33.2 ms
Wall time: 4.41 s


[ArrowRow({'id': 667,
           'ssn': 'anonomized-708296665',
           'name': 'anonomized-jjAdsa',
           'amount': 10005.0,
           'interest': 0.30000000000000004,
           'state': 'CA',
           'marital_status': 'single',
           'property': 'house',
           'dependents': 4,
           'defaulted': 1,
           'gender': 'M'})]

Let's try a `.map_batches()`, which is vectorized. We should expect faster execution. 

*Question: Why the `.map()` returned `ArrowRow` and `.map_batches` returned `PandasRow`*?

Because `map_batch(..., batch_format='native')` promotes it to Pandas DataFrame. You can
promote it to [other formats](https://docs.ray.io/en/master/data/package-ref.html#ray.data.Dataset.map_batches) such as `pyarrow`. The default is `native`, which is pandas.

In [20]:
%%time
arrow_ds.map_batches(lambda df: df[df["amount"] > 10000]).take(1)

Read->Map_Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 14.86it/s]

CPU times: user 73.3 ms, sys: 23.7 ms, total: 97 ms
Wall time: 411 ms


[PandasRow({'id': 667,
            'ssn': 'anonomized-708296665',
            'name': 'anonomized-jjAdsa',
            'amount': 10005.0,
            'interest': 0.30000000000000004,
            'state': 'CA',
            'marital_status': 'single',
            'property': 'house',
            'dependents': 4,
            'defaulted': 1,
            'gender': 'M'})]

You can see that `.map_batches()` is a lot faster than row based. So for large datasets use 
`.map_batches()`.

Let's try a filter operation: both per row operation and per block as vectorized

In [21]:
%%time
arrow_ds.filter(lambda x: x["amount"] > 10000.00 and x["state"] == 'CA').take(1)

Read->Filter: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  2.95it/s]

CPU times: user 20.2 ms, sys: 9.61 ms, total: 29.8 ms
Wall time: 1.71 s


[ArrowRow({'id': 667,
           'ssn': 'anonomized-708296665',
           'name': 'anonomized-jjAdsa',
           'amount': 10005.0,
           'interest': 0.30000000000000004,
           'state': 'CA',
           'marital_status': 'single',
           'property': 'house',
           'dependents': 4,
           'defaulted': 1,
           'gender': 'M'})]

In [22]:
%%time
arrow_ds.map_batches(lambda df: df[(df["amount"] > 10000) & (df["state"] == "CA")]).take(1)

Read->Map_Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 31.77it/s]

CPU times: user 26 ms, sys: 9.06 ms, total: 35.1 ms
Wall time: 172 ms


[PandasRow({'id': 667,
            'ssn': 'anonomized-708296665',
            'name': 'anonomized-jjAdsa',
            'amount': 10005.0,
            'interest': 0.30000000000000004,
            'state': 'CA',
            'marital_status': 'single',
            'property': 'house',
            'dependents': 4,
            'defaulted': 1,
            'gender': 'M'})]

Use `groupBy` state and compute the count

In [23]:
results = arrow_ds.groupby("state").count()

Shuffle Reduce: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 1001.36it/s]


In [24]:
results.show()

{'state': 'AZ', 'count()': 100313}
{'state': 'CA', 'count()': 99966}
{'state': 'NM', 'count()': 100288}
{'state': 'NV', 'count()': 99696}
{'state': 'OR', 'count()': 99590}
{'state': 'TX', 'count()': 100287}
{'state': 'UT', 'count()': 99992}
{'state': 'WA', 'count()': 99868}


Get the max of certain columns

In [25]:
results = arrow_ds.max(["amount", "interest", "dependents"])
results

Shuffle Reduce: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 358.27it/s]


ArrowRow({'max(amount)': 12000000.0,
          'max(interest)': 0.5,
          'max(dependents)': 5})

### Accessing datasets using batches or iterating by rows

Datasets can be passed to Ray tasks or actors and iterated over with `.iter_batches()` or `.iter_rows()`. (This does not incur a copy, since the blocks of the Dataset are passed by reference as Ray objects.) Splitting data as shards and passing to individual Ray Actors to process shards is a common Ray pattern used in distributed training with Ray actors.

Let's examine how we can process a list of shards with a `BatchWorker` Actor  in a distributed fashion

<img src="images/batch_worker.jpg" width="60%" height="30%">

A Ray actor `BatchWorker` working through shards in a batch size of 1024.

In [26]:
@ray.remote
class BatchWorker:
    def __init__(self, rank):
        self.rank = rank         # this could be rank of CPU/GPU or worker id
        self.processed = 0       # how much was processed
    
    @ray.method(num_returns=2)   # we want to return a tuple
    def process_shard_list(self, shard: ray.data.Dataset) -> tuple:
        for batch in shard.iter_batches(batch_size=1024):
            # here you could do something with the batch such as feature
            # preprocessing, minor transformation and then
            # save as a parquet file 
            self.processed = self.processed + len(batch)
        # return items processed, worker id
        return (self.processed, self.rank)     

#### Create batch workers as Ray actors
Each actor will get a shard (list of rows) to work on. We split
our dataset `arrow_ds` into five shards. Each `BatchWorker` gets a shard.

`.split`() splits shards across these batch of workers by using the `locality_hints`.

In [27]:
# Comprehension list to create five actors, each a BatchWorker
batch_workers = [BatchWorker.remote(i) for i in range(1, 6)]

# Split into five shards, each one for an actor and co-locate 
# the blocks of the dataset with each actor to maximize data locality.
shards = arrow_ds.split(len(batch_workers), locality_hints=batch_workers)

print(f"Shard row: {shards[0]}")
print(f"Number of shards:{len(shards)}")
print(f"Number of shard workers:{len(batch_workers)}")

Read progress: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 110.24it/s]

Shard row: Dataset(num_blocks=1, num_rows=160000, schema={id: int64, ssn: string, name: string, amount: double, interest: double, state: string, marital_status: string, property: string, dependents: int64, defaulted: int64, gender: string})
Number of shards:5
Number of shard workers:5



(_do_write pid=47064) E0814 15:18:28.922362000 6144749568 chttp2_transport.cc:1111]          Received a GOAWAY with error code ENHANCE_YOUR_CALM and debug data equal to "too_many_pings"


### Launch `BatchWorker` actors

Process each shard. Each `BatchWorker.process_shard_list()` returns a object RefID with a tuple as its value. What we get from this comprehension is a list objectRefs as tuples.

In [28]:
object_refs = [w.process_shard_list.remote(s) for w, s in zip(batch_workers, shards)]
object_refs, len(object_refs)

([[ObjectRef(e0513aa905843ccff55f6ffd7e97f4fc65c97bdc0100000001000000),
   ObjectRef(e0513aa905843ccff55f6ffd7e97f4fc65c97bdc0100000002000000)],
  [ObjectRef(5f70e045687d2f9a81ceea8b856b4b7457df42a80100000001000000),
   ObjectRef(5f70e045687d2f9a81ceea8b856b4b7457df42a80100000002000000)],
  [ObjectRef(a4dc031465f905f8881673411dfd28004e644af50100000001000000),
   ObjectRef(a4dc031465f905f8881673411dfd28004e644af50100000002000000)],
  [ObjectRef(9e7872a82e7456d9b1c33c160a4696d69b8e13830100000001000000),
   ObjectRef(9e7872a82e7456d9b1c33c160a4696d69b8e13830100000002000000)],
  [ObjectRef(cd25e647a728676b3dd45d790e057bccbb5b90bf0100000001000000),
   ObjectRef(cd25e647a728676b3dd45d790e057bccbb5b90bf0100000002000000)]],
 5)

Fetch the values from the returned list of ObjectRefs, which is a tuple of (batch_size, worker_rank).

In [29]:
values = [ray.get(ref) for ref in object_refs]
values

[[160000, 1], [160000, 2], [160000, 3], [160000, 4], [160000, 5]]

### Creating and using Ray dataset pipelines

What are dataset pipelines and how they are different from Ray datasets? 

Datasets perform transformation or operations eagerly or synchronously, whereas [DataPipelines](https://docs.ray.io/en/latest/data/package-ref.html#datasetpipeline-api) can execute in an overlapped pipeline executions. For example, if you had operations that require reading from a file, transforming data, and then doing some minor feature engineering, these operations can be executed in a normal pipeline fashion. This allows for the overlapped execution of data input (e.g., reading files), computation (e.g., feature preprocessing), and training (e.g., distributed ML training). 

<img src="images/pipeline_window.jpg" width="60%" height="30%">

#### Two ways to create `DatasetPipeline`

A `DatasetPipeline` can be constructed in two ways: either by pipelining the execution of an existing Dataset (via `Dataset.window`) or generating repeats of an existing Dataset (via `Dataset.repeat`). 

Let's have a go at it with the first method and see what we can do with our simple and synthetic data from above using `Dataset.window`.


### Using Dataset.window

Create simple functions or operations to be executed in a overlapped manner in the pipeline. These functions are simple to illustrate a point. But they can be complex for a particular use case.

In [30]:
def divide_row_value(row, n) -> int:
    return round(row / n)

In [31]:
def double_row_value(row, n) -> int:
    return row * n

In [32]:
def modulo_row_value(row , n) -> int:
    return row % random.randint(1, n)

#### Create a window based pipeline
With each window of 50 blocks. 

In [33]:
# Use our original simple dataset from above with 1K rows in integer
ds_pipe = ds.window(blocks_per_window=50)
ds_pipe

DatasetPipeline(num_windows=1, num_stages=2)

### Applying overlapping transforms to pipelines

#### Example 1:
This adds various overlapping functions with our simple dataset from above

In [34]:
ds_pipe = ds_pipe.map(lambda row: divide_row_value(row, 2))
ds_pipe = ds_pipe.map(lambda row: double_row_value(row, 3))
ds_pipe = ds_pipe.map(lambda row: modulo_row_value(row, 4))
print(ds_pipe)

DatasetPipeline(num_windows=1, num_stages=5)


#### Iterate our pipeline


In [35]:
results=[]
for row in ds_pipe.iter_rows():
    results.append(row)
# print(f"Results from each pipeline map function:{results}")
print(f"Total value of the results: {sum(results)}")

Stage 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.73it/s]

Total value of the results: 393


#### Example 2

Let's try a `Datapipeline` with our synthetic data *Homewowners* generated from above

In [36]:
# Define a function to count or return based on the condition
def count_state(row, state) -> int:
    return 1 if row['state'] == state and row["defaulted"] else 0

In [37]:
# Create a window-based pipeline
arrow_ds_pipe = arrow_ds.window(blocks_per_window=50)
arrow_ds_pipe

DatasetPipeline(num_windows=1, num_stages=2)

In [38]:
# Add the first stage to the pipeline
arrow_ds_pipe = arrow_ds_pipe.map(lambda row: count_state(row, "CA"))
arrow_ds_pipe

DatasetPipeline(num_windows=1, num_stages=3)

In [39]:
# Execute the pipeline 
results=[]
for row in arrow_ds_pipe.iter_rows():
    results.append(row)
print(f"Total rows for CA state and defaulted loans rows: {sum(results)}")

Stage 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.90s/it]

Total rows for CA state and defaulted loans rows: 50014


## Ingesting data into Model Trainers
Let's define a toy `Trainer` actor that takes our synthetic data and trains the model and returns loss for that trainer. This is a common pattern in Ray libraries for distributing data to Trainers in a Ray cluster.

<img src="images/trainer_worker.jpg" width="60%" height="30%">

In [40]:
# Our dummy model computing loss or accuracy
import time

def model(input):
    # do some training and compute the loss
    # ....
    # time.sleep(0.005)
    return random.uniform(0, 1)

@ray.remote
class Trainer:
    def __init__(self, rank, model):
        self.rank = rank
        self.model = model
        self.loss = 0.0
        
    def train(self, shard:ray.data.Dataset) -> float:
        for epoch in range(1,21):
            for batch in shard.iter_batches(batch_size=1024):
                output = self.model(batch)
                self.loss = output 
            if epoch % 5 == 0:
                print(f'rank: {self.rank} epoch: {epoch}, loss: {self.loss:.3f}')
        return self.loss

#### Create five trainers, each with a copy of the model and each training on its respective shard

In [41]:
trainers = [Trainer.remote(i, model) for i in range(1, 6)]
trainers

[Actor(Trainer, 4fc591febaabb850534acb2601000000),
 Actor(Trainer, 9db88fe04d232f762bf2e70a01000000),
 Actor(Trainer, 6ac1380e1ca535d6e6a2c5e701000000),
 Actor(Trainer, 6e909dab8ebdbab8e35d435801000000),
 Actor(Trainer, 43c636d6a8721cc047d7891301000000)]

#### Split the shards across all trainers

In [42]:
shards = arrow_ds.split(n=len(trainers), locality_hints=trainers)
shards

Read progress: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  7.06it/s]


[Dataset(num_blocks=1, num_rows=160000, schema={id: int64, ssn: string, name: string, amount: double, interest: double, state: string, marital_status: string, property: string, dependents: int64, defaulted: int64, gender: string}),
 Dataset(num_blocks=1, num_rows=160000, schema={id: int64, ssn: string, name: string, amount: double, interest: double, state: string, marital_status: string, property: string, dependents: int64, defaulted: int64, gender: string}),
 Dataset(num_blocks=1, num_rows=160000, schema={id: int64, ssn: string, name: string, amount: double, interest: double, state: string, marital_status: string, property: string, dependents: int64, defaulted: int64, gender: string}),
 Dataset(num_blocks=1, num_rows=160000, schema={id: int64, ssn: string, name: string, amount: double, interest: double, state: string, marital_status: string, property: string, dependents: int64, defaulted: int64, gender: string}),
 Dataset(num_blocks=1, num_rows=160000, schema={id: int64, ssn: string, 

#### Launch our trainers in a distributed fashion

This will run across the cluster. Check the dashbard to see five actors launched. On a cluster, they will be on five different nodes, whereas on a single node on  
five different cores.

In [43]:
object_refs = [t.train.remote(s) for t, s in zip(trainers, shards)]

(Trainer pid=49104) rank: 5 epoch: 5, loss: 0.208
(Trainer pid=49100) rank: 1 epoch: 5, loss: 0.954
(Trainer pid=49103) rank: 4 epoch: 5, loss: 0.869
(Trainer pid=49102) rank: 3 epoch: 5, loss: 0.852
(Trainer pid=49101) rank: 2 epoch: 5, loss: 0.817
(Trainer pid=49104) rank: 5 epoch: 10, loss: 0.306
(Trainer pid=49100) rank: 1 epoch: 10, loss: 0.943
(Trainer pid=49103) rank: 4 epoch: 10, loss: 0.766
(Trainer pid=49102) rank: 3 epoch: 10, loss: 0.590
(Trainer pid=49101) rank: 2 epoch: 10, loss: 0.460
(Trainer pid=49104) rank: 5 epoch: 15, loss: 0.549
(Trainer pid=49100) rank: 1 epoch: 15, loss: 0.744
(Trainer pid=49103) rank: 4 epoch: 15, loss: 0.054
(Trainer pid=49102) rank: 3 epoch: 15, loss: 0.760
(Trainer pid=49101) rank: 2 epoch: 15, loss: 0.383
(Trainer pid=49104) rank: 5 epoch: 20, loss: 0.099
(Trainer pid=49100) rank: 1 epoch: 20, loss: 0.919
(Trainer pid=49103) rank: 4 epoch: 20, loss: 0.140
(Trainer pid=49102) rank: 3 epoch: 20, loss: 0.700
(Trainer pid=49101) rank: 2 epoch: 2

In [44]:
ray.get(object_refs)

[0.9188967640709271,
 0.26947766649301796,
 0.699599068947067,
 0.14002369661670944,
 0.09855784439006066]

In [45]:
ray.shutdown()

### Exercises

Use [Ray data API](https://docs.ray.io/en/latest/data/package-ref.html) for reference:

 1. Write some simple transformers, filters, and aggregators with our synthetic data. For example:
  * use [`.add_column()`](https://docs.ray.io/en/latest/data/package-ref.html) to add an `age` column
  * filter by gender == 'U'
  * aggregate (or groupby `property`) and count each. 
 2. Add additional pipleline stages function `def count_tx(...)` with our synthetic data. For example, count all people in state of `TX` who are `married` and `defaulted`.

### Homework

So far we have covered the basics of Ray Datasets. There are advanced topics that you can now explore since you know the basics. Below is a list of tasks you will want to work through at home.

1. Re-write all `map_batches()` tasks transformations using [`ActorPoolStrategy` pool](https://docs.ray.io/en/latest/data/transforming-datasets.html#compute-strategy)
2. Work through the [NYC example tutorial](extra/ray_data_nyc.ipynb). This explores how you use `.map_batches()` for filtering and map operations and Parquet push-down predicates for projection.
3. Peruse the user guides for advanced examples in [data transformation](https://docs.ray.io/en/master/data/transforming-datasets.html#transforming-datasets) and [ML preprocessing](https://docs.ray.io/en/master/data/dataset-ml-preprocessing.html#datasets-ml-preprocessing).
4. Read how to do large scale [ML ingest](https://docs.ray.io/en/master/data/examples/big_data_ingestion.html).
5. Check out the advanced [pipeline usage](https://docs.ray.io/en/latest/data/advanced-pipelines.html#).

### Next Step

In Ray 2.0 we are introducing Ray AI Runtime (AIR) beta, an end-to-end machine learning toolkit. 
Let's take a peek into all its components and how you can use it as our [final lesson](ex_08_ray_air_in_ten_minutes.ipynb).

### References

1. [Ray Data Documentation](https://docs.ray.io/en/latest/data/dataset.html)
2. [Ray Data Webinar Talk](https://www.anyscale.com/events/2022/02/23/ray-datasets-scalable-data-preprocessing-for-distributed-ml)

📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
⬅️ [Previous notebook](./ex_06_ray_api_calls.ipynb) <br>

Done! 🍻